In [ ]:
from lib.correlation_integral import LongSpatial02CI
from models.lib.firstorderode.base import RungeKutta
from models.lib.firstorderode.lorenz import Lorenz63
from models.lib.firstorderode.roessler import Roessler76
from models.lib.firstorderode.sprott import SprottAttractors
from models.lib.toy_models import LinaJL
from reservoir_helper import get_lorenz63_data_and_reservoir

dt_02 = True
n_nodes = 10
i_model = 4

#get_lorenz63_data_and_reservoir = lambda seed, n_nodes: get_lorenz63_data_and_reservoir(seed, n_nodes, Model)
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from lib.attractor_deviation import AttractorDeviation
from lib.attractor_inclusion import AttractorInclusion
from lib.correlation_integral import LongSpatialCI, SpatialCI
from lib.distribution_deviation import DistributionDeviation
from lib.local_step_ahead import LocalStepAheadPredictability
from lib.total_variation import TotalVariation

if dt_02:
    models = []
    for ODEClass in [* SprottAttractors[1:4], Roessler76]:
        models += lambda seed = None, n_buffer = 10000, ODEClass=ODEClass: RungeKutta(ODEClass(seed), 2e-2, 10, n_buffer),
    models += lambda seed = None, n_buffer = 10000: RungeKutta(Lorenz63(s=10, r=28, b=8 / 3, seed=seed,), odeint_dt=1e-3, step_size=20, n_buffer=n_buffer,),
    CI = LongSpatial02CI
else:
    models = []
    for ODEClass in [* SprottAttractors[1:4], Roessler76]:
        models += lambda seed = None, n_buffer = 10000, ODEClass=ODEClass: RungeKutta(ODEClass(seed), 2e-2, 50, n_buffer),
    models += lambda seed = None, n_buffer = 10000: RungeKutta(Lorenz63(s=10, r=28, b=8 / 3, seed=seed,), odeint_dt=1e-3, step_size=100, n_buffer=n_buffer,),
    CI = LongSpatialCI

In [ ]:

from numpy import absolute

bounded = lambda pre: not (absolute(pre) > 2).any()
last = 5
osc = lambda pre, last=3: not (absolute(pre[-last:-1] - pre[-1]) < 5e-3).all() 

In [ ]:
from lib.attractor_deviation import AttractorDeviation
from lib.attractor_inclusion import AttractorInclusion
from lib.correlation_integral import LongSpatialCI, SpatialCI
from lib.distribution_deviation import DistributionDeviation
from lib.local_step_ahead import LocalStepAheadPredictability

lcis = list(map(
    lambda x: CI(f"data/heikki_long{"_dt_02" if dt_02 else ""}/" + x + f"_{CI.__name__}_ndata_300_nmul_100_seed_0.npz"), (
        map(lambda x: x().ode.__class__.__name__, models)
    ) 
))

adevs = list(map(
    lambda x: AttractorDeviation("data/hist/hist_" + x + ".npz"), (
        map(lambda x: x().ode.__class__.__name__, models)
    ) 
))

ddevs = list(map(
    lambda x: TotalVariation("data/hist/hist_" + x + ".npz"), (
        map(lambda x: x().ode.__class__.__name__, models)
    ) 
))

aincs = list(map(
    lambda x: AttractorInclusion("data/hist/hist_" + x + ".npz"), (
        map(lambda x: x().ode.__class__.__name__, models)
    ) 
))

plot_hist_measure = list(map(
    lambda x: TotalVariation("data/hist/hist_" + x + ".npz", resolution=64), (
        map(lambda x: x().ode.__class__.__name__, models)
    ) 
))

In [ ]:
from multiprocessing import Pool
from numpy.random import default_rng
from tqdm import tqdm
from lib.rmse import rmse
from lib.valid_prediction_time import valid_prediction_time
from scipy.stats import chi2
from numpy.random import SeedSequence

if dt_02:
    models = []
    for ODEClass in [* SprottAttractors[1:4], Roessler76]:
        models += lambda seed = None, n_buffer = 10000, ODEClass=ODEClass: RungeKutta(ODEClass(seed), 2e-2, 10, n_buffer),
    models += lambda seed = None, n_buffer = 10000: RungeKutta(Lorenz63(s=10, r=28, b=8 / 3, seed=seed,), odeint_dt=1e-3, step_size=20, n_buffer=n_buffer,),
    CI = LongSpatial02CI
else:
    models = []
    for ODEClass in [* SprottAttractors[1:4], Roessler76]:
        models += lambda seed = None, n_buffer = 10000, ODEClass=ODEClass: RungeKutta(ODEClass(seed), 2e-2, 50, n_buffer),
    models += lambda seed = None, n_buffer = 10000: RungeKutta(Lorenz63(s=10, r=28, b=8 / 3, seed=seed,), odeint_dt=1e-3, step_size=100, n_buffer=n_buffer,),
    CI = LongSpatialCI

def get_trajectory(inputs):
    spectral_radius, i_model, seed = inputs
    measures = []
    measures_ref = []
    names = []
    rng = default_rng(seed)

    weights_internal = np.eye(n_nodes)


    predictions_closed_loop, predictions_open_loop, data, normalizer, model, train, open_loop, closed_loop = get_lorenz63_data_and_reservoir(seed, n_nodes, models[i_model], spectral_radius=spectral_radius, regulizer=0*(1e-7 if dt_02 else 5e-6), weights_internal = weights_internal, g_in =(0.4 if dt_02 else 1) )
    
    compare = predictions_closed_loop[:-1]
        
    measures += [
    bounded(predictions_closed_loop), # type: ignore
    osc(predictions_closed_loop),
    (valid_prediction_time(data[closed_loop][1:], predictions_closed_loop[:-1])),
    rmse(data[closed_loop][1:], predictions_closed_loop[:-1])**2
    ]

    names += [
    ("bounded"), # type: ignore
    ("ozillating"),
    ("valid_prediciton_time"),
    ("mse")
    ]

    chiout = lcis[i_model](compare, rng, unnormalizer=normalizer.unnormalize)

    measures += 1-aincs[i_model](compare, normalizer=normalizer),
    measures += adevs[i_model](compare, normalizer=normalizer),
    measures += ddevs[i_model](compare, normalizer=normalizer),

    measures += chiout[-1][0],
    names += "Heikki",

    measures_ref += 1-aincs[i_model](data[closed_loop], normalizer=normalizer),
    measures_ref += adevs[i_model](data[closed_loop], normalizer=normalizer),
    measures_ref += ddevs[i_model](data[closed_loop], normalizer=normalizer),
    chiout = lcis[i_model](data[closed_loop], rng, unnormalizer=normalizer.unnormalize)
    measures_ref += chiout[-1][0],
    

    names += ("AIncs", "Adev", "Ddev")
            
    return predictions_closed_loop, data[closed_loop], measures, measures_ref



In [ ]:
models2 = [*SprottAttractors[1:4], Roessler76, Lorenz63]

data20 = np.load(f"results/spectral_radius_n_nodes_{20}/uncoupled_single_0.4{"" if not dt_02 else "_dt_02"}_{models2[i_model].__name__}.npz")
data20["measures"].shape

ss = SeedSequence(0)
ss.spawn(data20["measures"].shape[1])
seeds = np.array(ss.spawn(data20["measures"].shape[1]))

In [ ]:
data = data20["measures"].squeeze()#
print(data.shape)
idx = np.all(data[:, :2] == 1, axis=-1)
data = data[idx, 4:]
data = data[:, [3, 1, 2, 0]]
data[:, 0] = 1- data[:, 0]
seeds = seeds[idx]
print(data.shape)

In [ ]:
from mpl_toolkits.mplot3d import proj3d

ax = plt.subplot(111, projection="3d")
proj = lambda x, ax=ax: proj3d.proj_transform(*x.T, ax.get_proj())[:2]

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors

################## colormap ###################

cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 116./255., 116./255.),
                   (0.5, 1., 1.),
                   (1.0, 121./255., 121./255.)),

         'blue':  ((0.0, 122./255., 122./255.),
                   (0.5, 1., 1.0),
                   (1.0, 0.0, 0.0))
        }
       

TUI = LinearSegmentedColormap('TUI', cdict)
norm = mcolors.Normalize(vmin=0, vmax=1)  

In [ ]:
n = 10
def worker(i):
    return get_trajectory((0.4, 4, seeds[i]))

index = 0
succesful_tries = []
pbar = tqdm(total=n**2)

with Pool(48) as p:
    while len(succesful_tries) < n**2:
        for data in p.map(worker, range(index, index+48)):
            if np.array(data[2][:2]).all() and len(succesful_tries) < n**2:
                succesful_tries.append(data)
                pbar.update(1)
        index += 48

all_rejected_measures = list(map(np.array, zip(*succesful_tries)))
pbar.close()

In [ ]:
len(succesful_tries)

In [ ]:
fig, axs = plt.subplots(n, n, dpi=300)
for i in range(n):
    axs[0, i].set_title(chr(65+i))
    axs[i, 0].set_ylabel(i)
for ax, (prediction, data, measure, _) in zip(axs.flatten(), zip(*all_rejected_measures)):
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

    # Hide the spines (borders)
    for spine in ['top', 'bottom', 'left', 'right']:
        ax.spines[spine].set_visible(False)
    
    ax.plot(*proj(data), lw=0.1, rasterized=True, color=TUI(norm(1)), alpha=0.8)
    ax.plot(*proj(prediction), lw=0.1, rasterized=True, color=TUI(0), alpha=0.8)
    # ax = axs[1]
    # ax.hist2d(*proj(prediction), rasterized=True, bins=64, cmap="Grays")
    # plt.show()
plt.savefig("pictures/illustrations/grid_timeseries.pdf")

In [ ]:
fig, axs = plt.subplots(n, n, dpi=300)
from matplotlib import colors
# color pallet
cmap = plt.cm.Greys.copy()
cmap.set_over('#fc03f8')     # For values > vmax
norm_colorbar = colors.Normalize(0, 13.2, clip=False)

for i in range(n):
    axs[0, i].set_title(chr(65+i))
    axs[i, 0].set_ylabel(i)
for ax, (prediction, data, measure, _) in zip(axs.flatten(), zip(*all_rejected_measures)):
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

    # Hide the spines (borders)
    for spine in ['top', 'bottom', 'left', 'right']:
        ax.spines[spine].set_visible(False)
    
    _, _, _, hist = ax.hist2d(*proj(prediction), rasterized=True, bins=64, cmap=cmap, norm=norm_colorbar)
    
cbar = fig.colorbar(hist, ax=axs.ravel().tolist(), extend="max", cmap=cmap)
cbar.set_label("Counts", size=12)
cbar.set_ticklabels(cbar.get_ticks(), size=12)
mappable = cbar.mappable
mappable.set_clim(0.0, 13.2)
cbar.update_normal(mappable)
cbar.outline.set_linewidth(0.3)
plt.savefig("pictures/illustrations/grid_reservoir.pdf")

In [ ]:
fig, axs = plt.subplots(n, n, dpi=300)
mx = 0
for i in range(n):
    axs[0, i].set_title(chr(65+i))
    axs[i, 0].set_ylabel(i)
for ax, (prediction, data, measure, _) in zip(axs.flatten(), zip(*all_rejected_measures)):
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

    # Hide the spines (borders)
    for spine in ['top', 'bottom', 'left', 'right']:
        ax.spines[spine].set_visible(False)
    
    counts, _, _, hist = ax.hist2d(*proj(data), rasterized=True, bins=64, cmap=cmap, norm=norm_colorbar)
    mx = mx if mx >= np.max(counts) else np.max(counts)
cbar = fig.colorbar(hist, ax=axs.ravel().tolist(), extend="max", cmap=cmap)
cbar.set_label("Counts", size=12)
cbar.set_ticklabels(cbar.get_ticks(), size=12)
mappable = cbar.mappable
mappable.set_clim(0.0, 13.2)
cbar.update_normal(mappable)
cbar.outline.set_linewidth(0.3)
print(mx)
plt.savefig("pictures/illustrations/grid_lorenz63.pdf")

In [ ]:


def get_index(number: int, letter: str):
    index_x = ord(letter.upper()) - 65
    index_y = number
    return index_x + index_y * n

to_plot = [
    get_index(1, "A"),
    get_index(7, "D"),
    get_index(4, "D"),
    get_index(5, "E"),
    get_index(5, "J"),
    get_index(8, "B"),
]

cm = 1/2.54
fig, axs = plt.subplots(3, len(to_plot), figsize=(21*cm-5*cm, 4.5*cm), dpi=300, height_ratios=[1, 1, 0.5])


from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
from matplotlib import colors

################## colormap ###################

cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 116./255., 116./255.),
                   (0.5, 1., 1.),
                   (1.0, 121./255., 121./255.)),

         'blue':  ((0.0, 122./255., 122./255.),
                   (0.5, 1., 1.0),
                   (1.0, 0.0, 0.0))
        }
       

TUI = LinearSegmentedColormap('TUI', cdict)
norm = mcolors.Normalize(vmin=0, vmax=1)  

for i, (ax_row, index) in enumerate(zip(axs.T, to_plot)):
    (prediction, data, measure, _) = list(map(lambda x: x[index], all_rejected_measures))

    if i == 0:
        # color pallet
        cmap = plt.cm.Greys.copy()
        cmap.set_over('#fc03f8')     # For values > vmax
        # find common colorbar
        max_counts, x_edges, y_edges = np.histogram2d(*proj(prediction), bins=64)
        norm_colorbar = colors.Normalize(0, 13.2, clip=False)
        _, _, _, hist = ax.hist2d(*proj(prediction), rasterized=True, bins=[x_edges, y_edges], norm=norm_colorbar, cmap=cmap)
        #cbar = miniplot_grid.cbar_axes[0].colorbar(hist, extend="max", cmap=cmap)
        #cbar.set_label("Counts", size=12)

    for ax in ax_row:
        ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
        ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

        # Hide the spines (borders)
        for spine in ['top', 'bottom', 'left', 'right']:
            ax.spines[spine].set_visible(False)
    
    ax_row[0].plot(*proj(data), lw=0.1, rasterized=True, color=TUI(norm(1)), alpha=0.8)
    ax_row[0].plot(*proj(prediction), lw=0.1, rasterized=True, color=TUI(0), alpha=0.8)
    ax_row[1].hist2d(*proj(prediction), rasterized=True, bins=64, norm=norm_colorbar, cmap=cmap)
    ax_row[1].set_xlim(ax_row[0].get_xlim())
    ax_row[1].set_ylim(ax_row[0].get_ylim())

    ax_row[2].axis([0, 1, 0, 4])
    for i, (value, name) in enumerate(zip(measure[4:], ("AExc", "ADev", "TVar", "GCI"))):
        if name == "ADev" or name == "GCI":
            text = f"{value:.0f}"
        elif name == "TVar":
            text = f"{value:.3f}"
        else:
            text = f"{value:.2f}"
        ax_row[2].text(0.5, 1.3*i, text, horizontalalignment="left", fontsize=5, color='black')
        ax_row[2].text(0.2, 1.3*i, f"{name}: ", horizontalalignment="left", fontsize=5, color='black')
    # ax = axs[1]
    # ax.hist2d(*proj(prediction), rasterized=True, bins=64, cmap="Grays")
    # plt.show()
plt.tight_layout()
cbar = fig.colorbar(hist, ax=axs.ravel().tolist(), extend="max", cmap=cmap)
cbar.set_label("Counts", size=5)
cbar.set_ticklabels(cbar.get_ticks(), size=5)
mappable = cbar.mappable
mappable.set_clim(0.0, 13.2)
cbar.update_normal(mappable)
cbar.outline.set_linewidth(0.3)
plt.savefig("pictures/illustrations/bad_examples.pdf", bbox_inches='tight')